# EDA Gold Rate and forecast
---
This notebook contains Exploratory Data Analysis of Gold rates of six countries based on their currency: United States, Europe, Great Britian, India, UAE and China as well as forecasting using XGBoost.

### Information about the data
* date - Date
* USD - Price of gold in US dollars
* EUR - Price of gold in Euros
* GBP - Price of gold in Pounds
* INR - Price of gold in Rupees
* AED - Price of gold in Dirhams
* CNY - Price of gold in Chinese Yuans.

### Import Module

In [2]:
# Core
import sys
import pandas as pd
import numpy as np
import os
import time
import math
from datetime import date,datetime,timedelta

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objs as go
%matplotlib inline

### Reading input file

In [3]:
# Path of file to read
path = os.getcwd()

# Load data into pandas dataframe
daily = pd.read_csv(os.path.join(path, 'daily_gold_rate.csv'))
annual = pd.read_csv(os.path.join(path, 'annual_gold_rate.csv'))

---

## Explore the data

We first explore the data to make sense of them we will mainly looking at:
1. Understanding data and its type.
2. Check for missing or null.
3. Find potential outliers.
4. Find correlations.
5. Check for data skew.

### Prelim Observation

In [4]:
daily.head()

,Date,USD,EUR,GBP,INR,AED,CNY
0,1985-01-01,308.30,402.45,266.01,3825.85,1134.42,NaN
1,1985-01-02,305.50,401.49,266.00,3787.05,1134.42,NaN
2,1985-01-03,302.30,397.08,262.76,3718.88,1107.67,NaN
3,1985-01-04,303.15,397.15,262.58,3759.64,1118.21,NaN
4,1985-01-07,298.25,392.55,261.17,3741.23,1091.58,835.8


In [5]:
daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    9704 non-null   object 
 1   USD     9704 non-null   float64
 2   EUR     9704 non-null   float64
 3   GBP     9704 non-null   float64
 4   INR     9704 non-null   float64
 5   AED     9704 non-null   float64
 6   CNY     9700 non-null   float64
dtypes: float64(6), object(1)
memory usage: 530.8+ KB


In [6]:
daily.describe()

,USD,EUR,GBP,INR,AED,CNY
count,9704.000000,9704.000000,9704.000000,9704.000000,9704.000000,9700.000000
mean,754.900371,624.274441,499.861527,39440.737164,2772.562061,4987.779785
std,509.680400,414.592280,374.830318,38459.177898,1872.224258,3378.756103
min,252.800000,222.200000,157.150000,3687.170000,928.530000,809.630000
25%,353.987500,305.247500,219.522500,11598.695000,1298.505000,2189.372500
50%,425.450000,362.085000,259.370000,17038.720000,1562.390000,3311.720000
75%,1234.925000,1016.015000,823.062500,75974.875000,4535.797500,8097.082500
max,2067.150000,1874.560000,1572.810000,156815.690000,7592.950000,14355.530000


There appear to be some missing data in CNY.
We can assume that there is no trade at that moment or official record got lost in time. We will fill them with 0

In [7]:
daily.fillna(0)

,Date,USD,EUR,GBP,INR,AED,CNY
0,1985-01-01,308.30,402.45,266.01,3825.85,1134.42,0.00
1,1985-01-02,305.50,401.49,266.00,3787.05,1134.42,0.00
2,1985-01-03,302.30,397.08,262.76,3718.88,1107.67,0.00
3,1985-01-04,303.15,397.15,262.58,3759.64,1118.21,0.00
4,1985-01-07,298.25,392.55,261.17,3741.23,1091.58,835.80
...,...,...,...,...,...,...,...
9699,2022-03-07,1980.95,1821.48,1508.09,152468.77,7276.13,12516.24
9700,2022-03-08,2039.05,1874.56,1555.28,156815.69,7489.63,12874.97
9701,2022-03-09,1988.90,1800.64,1511.72,152282.61,7305.33,12562.39
9702,2022-03-10,1996.60,1812.04,1520.81,152350.57,7333.61,12622.21


In [8]:
annual.head()

,Date,USD,EUR,GBP,INR,AED,CNY
0,1978,193.44,124.73580,100.65449,NaN,NaN,NaN
1,1979,304.68,185.06142,143.09014,2481.90512,1163.93597,NaN
2,1980,614.50,378.19461,263.80259,4831.74943,2278.36795,NaN
3,1981,459.26,360.90588,227.01948,3966.64790,1691.03194,NaN
4,1982,375.30,345.59742,215.53232,3553.62680,1378.72895,NaN


In [9]:
annual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    44 non-null     int64  
 1   USD     44 non-null     float64
 2   EUR     44 non-null     float64
 3   GBP     44 non-null     float64
 4   INR     43 non-null     float64
 5   AED     43 non-null     float64
 6   CNY     37 non-null     float64
dtypes: float64(6), int64(1)
memory usage: 2.5 KB


In [11]:
annual.describe()

,Date,USD,EUR,GBP,INR,AED,CNY
count,44.000000,44.000000,44.000000,44.000000,43.000000,43.000000,37.000000
mean,1999.500000,692.055909,571.891313,450.575615,34026.445973,2585.786320,4950.991330
std,12.845233,486.226172,395.398741,357.907845,37476.310663,1783.571654,3377.322124
min,1978.000000,193.440000,124.735800,100.654490,2481.905120,995.506180,926.645180
25%,1988.750000,360.540000,302.387860,215.853185,6423.166690,1327.153290,2243.380520
50%,1999.500000,416.415000,359.622620,249.850690,12788.438900,1557.457900,3306.274890
75%,2010.250000,1176.175000,932.098448,761.273783,64728.547700,4379.206195,8283.158470
max,2021.000000,1798.610000,1549.044710,1378.980990,132967.875100,6606.550460,12205.481970


Again, some missing data we will deal with them the same as CNY daily.

In [12]:
annual.fillna(0)

,Date,USD,EUR,GBP,INR,AED,CNY
0,1978,193.44,124.73580,100.65449,0.00000,0.00000,0.00000
1,1979,304.68,185.06142,143.09014,2481.90512,1163.93597,0.00000
2,1980,614.50,378.19461,263.80259,4831.74943,2278.36795,0.00000
3,1981,459.26,360.90588,227.01948,3966.64790,1691.03194,0.00000
4,1982,375.30,345.59742,215.53232,3553.62680,1378.72895,0.00000
5,1983,423.66,439.89318,279.24936,4270.42821,1557.45790,0.00000
6,1984,360.78,425.91000,269.81553,4067.60433,1324.72272,0.00000
7,1985,317.30,394.08922,246.28501,3887.39270,1162.19778,926.64518
8,1986,367.85,351.34774,251.12483,4625.06990,1351.92474,1272.38854
9,1987,446.22,366.06607,272.49236,5761.91787,1641.52338,1659.60015


### Clean some data
Currently `Date` column is in incorrect datatype so we will correct that.

In [13]:
# Change to y/m/d date format
daily['Date'] = pd.to_datetime(daily['Date'], format="%Y-%m-%d")
annual['Date'] = pd.to_datetime(annual['Date'], format="%Y-%m-%d")

---
## Visualization

I want to create range slider for graph thoughout different time period.
I found that using `plotly.graph_objects` is very easy. Simply create dict based on what you needed.
As I will create for all countries I will create function for this task.

In [ ]:
# Function for create graph for currency

def plot_daily(currency):
    month = dict(count=1, label="1M", step="month", stepmode="backward")
    three_month = dict(count=3, label="3M", step="month", stepmode="backward")
    six_month = dict(count=6, label="6M", step="month", stepmode="backward")
    year = dict(count=1, label="1Y", step="year", stepmode="backward")
    three_year = dict(count=3, label="3Y", step="year", stepmode="backward")
    five_year = dict(count=5, label="5Y", step="year", stepmode="backward")
    alltime = dict(label="All time", step="all")
    fig = go.Figure()
    fig.addtrace(go.Scatter(x=daily['Date'], y=daily[f'{currency}']))

